In [1]:
from IPython.display import clear_output
%cd /content/drive/MyDrive/collab_sandbox/learn_DL/dive_into_dl/
# !pip install colabcode
clear_output()

# Pandas

https://pandas.pydata.org/docs/user_guide/10min.html#setting

# Titanic Kaggle

https://www.kaggle.com/c/titanic

In [71]:
import pandas as pd
import torch
import numpy as np
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

In [72]:
dataset_train = pd.read_csv("./data/TitanicKaggle/train.csv")
dataset_test = pd.read_csv("./data/TitanicKaggle/test.csv")

drop_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
dataset_train = dataset_train.drop(columns=drop_columns)

In [74]:
dataset_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [75]:
def encode_categorical(dataset, columns):
  dataset = dataset.copy(deep=True)
  dataset_cols = dataset.columns.values
  for column in columns:
    if column in dataset_cols:
      values_list = dataset[column].values.tolist()
      list_columns = sorted(list(map(str, list(set(values_list)))))
      list_columns_renamed = [f'{column}_{item}' for item in list_columns]
      dict_columns = {item: i for (i, item) in enumerate(list_columns)}
      
      for i, value in enumerate(values_list):
        value = str(value)

        ohe_hot = [0 for _ in range(len(list_columns))]
        ohe_hot[dict_columns.get(value)] = 1
        dataset.loc[i, list_columns_renamed] = ohe_hot
      dataset = dataset.drop(columns=[column],) 
  
  return dataset

# encode_categorical(dataset_train, ['Sex', 'Embarked']).head(3)

In [76]:
categorical_labels = ['Pclass', 'SibSp',  'Embarked', 'Sex', 'Parch',]
dataset_train = encode_categorical(dataset_train, categorical_labels)
dataset_train.head(5)

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Sex_female,Sex_male,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,0,22.0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,38.0,71.2833,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,26.0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,35.0,53.1000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,35.0,8.0500,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
def standardize_data(dataset, exclude_columns):
  dataset = dataset.copy(deep=True)
  dataset_cols = list(set(dataset.columns.values) - set(exclude_columns))
  for column_name in dataset_cols:
    # column_name = "Age"
    column = dataset[column_name].to_numpy()
    nan_indexes = np.isnan(column)
    real_indexes = ~np.isnan(column)

    n = real_indexes.astype(int).sum()
    real_column = column[real_indexes]

    real_column_mean = real_column.sum() / n # or .mean()  
    real_column_var = (np.sum([(column[i]-real_column_mean)**2 if item else 0 for i, item in enumerate(list(real_indexes))]) / n)**(1/2)
    std_column = np.array([(column[i]-real_column_mean) / real_column_var if item else 0 for i, item in enumerate(list(real_indexes))])
    
    dataset.loc[:, column_name] = std_column 
  return dataset

dataset_train = standardize_data(dataset_train, ['Survived'])
dataset_train.head()

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Sex_female,Sex_male,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,0,-0.530377,-0.502445,-0.565685,-0.510152,0.902587,-1.465746,1.806421,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,-0.737695,0.737695,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
1,1,0.571831,0.786845,1.767767,-0.510152,-1.107926,-1.465746,1.806421,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,2.074505,-0.307562,-1.614710,-0.047431,1.355574,-1.355574,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
2,1,-0.254825,-0.488854,-0.565685,-0.510152,0.902587,0.682247,-0.553581,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,1.355574,-1.355574,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
3,1,0.365167,0.420730,1.767767,-0.510152,-1.107926,-1.465746,1.806421,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,1.355574,-1.355574,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
4,0,0.365167,-0.486337,-0.565685,-0.510152,0.902587,0.682247,-0.553581,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,-0.737695,0.737695,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352


In [38]:
dataset_train.iloc[0][['Survived']]
# some = dataset_train.iloc[0]

Survived    0.0
Name: 0, dtype: float64

In [34]:
# some.drop(labels=['Survived'])

https://pytorch.org/docs/stable/generated/torch.nn.Module.html

In [93]:
class LinearRegressionDirty(nn.Module):
  def __init__(self, 
                features=None,
                targets=None
    ):
    super(LinearRegressionDirty, self).__init__()
    self.W = torch.nn.Parameter(
        torch.normal(0, 0.01, 
        size=(features, targets), 
        requires_grad=True))
    
    self.b = torch.nn.Parameter(
        torch.normal(0, 0.01,
        size=(1,targets),
        requires_grad=True))
    
  
  
  @staticmethod
  def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

  def softmax(self, x):
    x = torch.exp(x)
    x_sum = x.sum(1, keepdim=True)
    return x / x_sum 

  def forward(self, x):
    # print(self.W.shape, x.shape, self.b.shape)
    x = torch.matmul(x, self.W) + self.b
    # print(x.shape)
    x = self.softmax(x)
    return x


model = LinearRegressionDirty(features=3, targets=2)
# print("model.parameters", list(model.parameters()))
model(torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [11, 12, 13]], dtype=torch.float32))

tensor([[0.4892, 0.5108],
        [0.4798, 0.5202],
        [0.4705, 0.5295],
        [0.4580, 0.5420]], grad_fn=<DivBackward0>)

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
class CustomPandasDataset(Dataset):
  def __init__(self, 
                dataset=None,
                target_columns=None
      ):
      self.dataset = dataset
      self.target_columns = target_columns

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      data_row = self.dataset.iloc[idx]
      target = torch.tensor(data_row[self.target_columns].values, dtype=torch.long).squeeze(0)
      data_row = data_row.drop(labels=self.target_columns)
      features = torch.tensor(data_row.values, dtype=torch.float32)

      return features, target 

temp_dataset = CustomPandasDataset(
    dataset=dataset_train,
    target_columns=['Survived']
)

temp_dataloader = DataLoader(
    dataset=temp_dataset, 
    batch_size=16, 
    shuffle=True
)

next(iter(temp_dataloader))
# temp_dataset

In [ ]:
dataset_train.iloc[0][['Survived']].values

In [124]:
class Trainer:
  def __init__(self, 
               model=None, 
               dataloader=None,
               optimizer_class=None,
               loss_func=None,
    ): 
    self.model = model
    self.dataloader = dataloader
    self.optimizer_class = optimizer_class
    self.optimizer = None
    self.loss_func = loss_func

    # self.unit_tests()

  def train(self, 
            is_unit_test=False,
            epochs=1,
            lr=0.01
    ):
    self.optimizer = self.optimizer_class(
        params=self.model.parameters(),
        lr=lr
    )

    for epoch in range(epochs):
      total_examples = 0
      total_loss = 0
      for features, targets in self.dataloader:
        self.optimizer.zero_grad()

        predicts = self.model(features)
        # print(predicts, targets)
        loss = self.loss_func(predicts, targets)
        total_loss += loss
        total_examples += len(targets)
        loss.backward()
        self.optimizer.step()
        
        if is_unit_test:
          break
      print(f"loss => {total_loss / total_examples}")
      
      if is_unit_test:
        break

  
  def unit_tests(self):
    print("run unit tests")
    self.train(
        is_unit_test=True,
        epochs=1,
        lr=0.01
    )
    print("Ok")


train_dataset = CustomPandasDataset(
    dataset=dataset_train,
    target_columns=['Survived']
)
train_dataloader = DataLoader(
    dataset=temp_dataset, 
    batch_size=16, 
    shuffle=True
)

# print(train_dataset[0])
features_size = len(train_dataset[0][0])
targets_size = 2 #len(train_dataset[0][1])
model=LinearRegressionDirty(
    features=features_size, 
    targets=targets_size
)

optimizer_class = torch.optim.SGD
loss_func = nn.CrossEntropyLoss()
# loss_func = LinearRegressionDirty.cross_entropy
trainer = Trainer(
  model=model, 
  dataloader=train_dataloader,
  optimizer_class=optimizer_class,
  loss_func=loss_func
)

In [127]:
trainer.train(epochs=10)

loss => 0.038993094116449356
loss => 0.037216708064079285
loss => 0.03612475469708443
loss => 0.03533726558089256
loss => 0.034798428416252136
loss => 0.03434450551867485
loss => 0.0340394452214241
loss => 0.03373677283525467
loss => 0.033546097576618195
loss => 0.03335462510585785


In [128]:
with torch.no_grad():
  total_acc = 0
  total_features = 0
  for features, targets in train_dataloader:
    infer_preds = trainer.model(features).argmax(-1)

    total_acc += (infer_preds == targets).type(torch.float32).sum()
    total_features += len(targets)
  
  print(f"acc: {total_acc / total_features} " )

acc: 0.7946127653121948 
